## Accuire Vehicle and Non-vehicle paths and split to train and test

In [2]:
from sklearn.model_selection import train_test_split
import glob
import numpy as np

vehicle_sets = {
    'vehicles/GTI_MiddleClose': [
        [0,32,50,55,78,89,100,110], # Strang things happen here, manually fix
        [126,136,156,161,171,181,191],
        [200,216,226,229,239,248,267,282,290,300,310,
         313,328,332,342,349,359,366,377,387,398,400,
         403,406,409,411,414,417,420]]
}

gti_vehicle_folders = ['./vehicles/GTI_MiddleClose',
               './vehicles/GTI_Far',
               './vehicles/GTI_Left',
               './vehicles/GTI_Right']

# Fraction of samples that will be allocated as test set.
test_size_ratio = 0.2

# I choose to take the first test_size_ratio part from all paths, to avoid
# getting similar photo of same car in both testing set and training set.
# If similar photo is in both training set and testing set, then it is easy
# to get good score on testing set, even though it is because of overfitting.
# A similar photo of same car can appear in eg. both GTI_Left and GTI_MiddleClose.
# By taking first photos from all folders, I avoid probability photo of car
# from one folder ends up in testing set, while a similar photo in the other
# folder ends up in the training set, because it is roughly same order of the
# the different cars in both folders.
car_paths_train = []
car_paths_test = []
for folder in gti_vehicle_folders:
    car_paths = glob.glob(folder + '/*.png')
    assert car_paths # Assert not empty
    car_paths = list(sorted(car_paths))
    
    num_test = int(test_size_ratio * len(car_paths))
    # Allocate first test_size_ratio part as test set.
    car_paths_test.extend(car_paths[:num_test])
    # Allocate rest to train set.
    car_paths_train.extend(car_paths[num_test:])

kitti_paths = glob.glob('vehicles/KITTI_extracted/*.png')
assert kitti_paths # Assert not empty

rand_state = np.random.randint(0, 100)
kitti_vehicle_train, kitti_vehicle_test = train_test_split(
    kitti_paths, test_size=test_size_ratio, random_state=rand_state)

car_paths_train.extend(kitti_vehicle_train)
car_paths_test.extend(kitti_vehicle_test)

# The images in non-vehicles also form runs of images, but they look quite
# different inside a run, so I just randomly split it.
noncar_paths = glob.glob('./non-vehicles/**/*.png')
assert noncar_paths # Assert not empty

noncar_paths_train, noncar_paths_test = train_test_split(
    noncar_paths, test_size=test_size_ratio, random_state=rand_state)

print('car_paths_train length:', len(car_paths_train))
print('car_paths_test length:', len(car_paths_test))

print('noncar_paths_train length:', len(noncar_paths_train))
print('noncar_paths_test length:', len(noncar_paths_test))

car_paths_train length: 7036
car_paths_test length: 1756
noncar_paths_train length: 7174
noncar_paths_test length: 1794


In [ ]:
#import glob
## Read in cars and notcars
#car_paths = glob.glob('./vehicles/**/*.png')
#assert car_paths # Assert not empty
#
#noncar_paths = glob.glob('./non-vehicles/**/*.png')
#assert noncar_paths # Assert not empty

In [3]:
# From course section: 'Search and Classify'
import matplotlib.image as mpimg
import numpy as np
import cv2
from skimage.feature import hog

# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features 
# NEED TO CHANGE bins_range if reading .png files with mpimg!
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)

In [4]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time
import sklearn
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog

def extract_features(imgpaths, *args, **kwargs):
    features = []
    for imgpath in imgpaths:
        img = cv2.imread(imgpath)
        # Convert from BGR to RGB for to be compatible with rest of the pipline.
        img = img[:,:,::-1]
        features.append(single_img_features(img, *args, **kwargs))
    return features


### TODO: Tweak these parameters and see how the results change.
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (32, 32) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [None, None] # Min and max in y to search in slide_window()

def call_extract_features(paths):
    return extract_features(paths, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

cars_train = call_extract_features(car_paths_train)
cars_test = call_extract_features(car_paths_test)
noncars_train = call_extract_features(noncar_paths_train)
noncars_test = call_extract_features(noncar_paths_test)

X_train = np.vstack((cars_train, noncars_train)).astype(np.float64)                        
X_test = np.vstack((cars_test, noncars_test)).astype(np.float64)                        

# Fit a per-column scaler.
# Only train it on train set.
X_scaler = StandardScaler().fit(X_train)

# Apply it to train set.
X_train = X_scaler.transform(X_train)

# Apply it to test set.
X_test = X_scaler.transform(X_test)

# Define the labels vector
y_train = np.concatenate((np.ones(len(cars_train)), np.zeros(len(noncars_train))))
y_test = np.concatenate((np.ones(len(cars_test)), np.zeros(len(noncars_test))))

# Shuffle doesn't hurt.
X_train,y_train = sklearn.utils.shuffle(X_train,y_train)
X_test,y_test = sklearn.utils.shuffle(X_test,y_test)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))



/home/mandermo/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 8460
4.7 Seconds to train SVC...
Test Accuracy of SVC =  0.9865
